# Process data

### Separate threads and posts

In [ ]:
import csv
import sys
import codecs
import os
import re
from difflib import SequenceMatcher
import itertools
import json
import codecs
import pandas as pd

#this is to extent the size of the reading csv cell
# csv.field_size_limit(sys.maxsize)
file_path='path_to_raw+data'
save_path_trash='path_to_save_data'


def UTC_separation(thread):
    posts_lst=[]
    posts=re.split(r'(\d\d:\d\d, \d+ \w+ \d\d\d\d \(UTC\))',thread)# split the posts based on the timestamp
    for i in range(0,len(posts)-1, 2):
        posts_lst.append(posts[i]+posts[i+1])# connects post with timestamp because the splot separates them
    
    return posts_lst

#to separate discussions with no title
def thread_without_title(temp, page_title, save_path_data):
    if ((temp[0:2]=='{{') and (('documentation' in temp) or ('Documentation' in temp))) or ((temp[0:2]=='{{') and (('Interwiki conflict' in temp) or ('interwiki conflict' in temp))) or ('#REDIRECT' in temp): # checks if the text includes only documentation, not discussions
         trash=pd.DataFrame({'page_title':[page_title], 'text':temp})
         trash.to_csv(str(save_path_trash),mode='a',index=False, header=not os.path.exists(str(save_path_trash)))
    else:
        thread_title='No subject'
        posts_lst = UTC_separation(temp)
        #store posts with the corresponding sibject
        df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
        #save to csv 
        df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))


def thread_with_title(titles, temp, page_title, save_path_data):
    for j in range(len(temp)):
        if (temp[j] in titles):
            if (j+1)==(len(temp)):
                thread_title=temp[j]
                posts_lst = []
                    #store posts with the corresponding sibject
                df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
                    #save to csv 
                df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))
            else:
                thread_title=temp[j]
                posts_lst = UTC_separation(temp[j+1])
                    #store posts with the corresponding sibject
                df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
                    #save to csv 
                df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))
        
    

#function to create a file with the separated posts  
def separate_discussions(text, page_title, save_path_data):

    titles = re.findall('==(.*)==', text)# find the titles in the discussion  
    titles=[s.replace('=', '') for s in titles]# removes unecessery = in the title
    
    
    #titles=list(set(titles)) 
    temp= text.split('==') # split the text based on == , includided in the title
    temp = [x for x in temp if x != ''] # stores the splited parts
    temp=[s.strip('=') for s in temp]# removes unecessery = in the title

    # ---Step 1: there is no title in the page
    if titles==[]: # check if there is title in the begining of the page
        thread_without_title(temp[0], page_title, save_path_data) 
    else:
        if temp[0] not in titles:
            thread_without_title(temp[0], page_title, save_path_data)# ---Step 2: the first thread do not have title 
            thread_with_title(titles, temp, page_title, save_path_data)# ignores the first if does not have a title and process the rest threads
        else:
            thread_with_title(titles, temp, page_title, save_path_data)# ---Step 3: a;; threads have titles 



file_lst=['items.csv','properties.csv']
    

for file in file_lst:
    save_path_data='path'+str(file) # path to save posts and threads
    data=pd.read_csv(str(file_path)+str(file), encoding='utf-8')# read file
    for row in range(len(data)):# read row by row (i.e., page by page)
        print(row)
        # print(data.loc[row,'page_title'])
        # print(data.loc[row,'text'])
        page_title=data.loc[row,'page_title']
        if isinstance(data.loc[row,'text'],str):# check of the row is nan. If it is nan the type is float not str
            separate_discussions(data.loc[row,'text'], page_title, save_path_data)
        else:
            empty=pd.DataFrame({'page_title':[page_title], 'text':'empty'})
            empty.to_csv(str(save_path_trash),mode='a',index=False, header=not os.path.exists(str(save_path_trash)))

### Extract username and timestamp

In [ ]:
import pandas as pd
import os
import re

# find usernmae based on [[User talk:]] pattern
def find_username_User_talk(text):
    pattern=re.compile(r"(\[)?(User:|User talk:|user:|user talk:|User Talk:|User_talk:|User_Talk:|user_talk:|Utente:|Usuário:|Utilisateur:|Utilizator:|Usuari Discussió:|Usuario discusión:|Usuario:|Usuario Discusi\u00f3n:|kullanıcı:|Kullan\u0131c\u0131:|:USER TALK:|:USER:|U:|u:|Benutzer:|Benutzer Diskussion:|Special:Contributions/|welcominguser=|Discussão:|Dyskusja_Wikipedysty:|사용자토론:)(.*?)(\||/|\])") # the regular expression I need to extract 
    usernames=pattern.finditer(text)
    #search for the username
    names=[]        
    for match in usernames:
        #print(match.group(2))
        names.append([match.group(3),match.end()])
        
    if len(names)>0:
        max_loc=max([sublist[1] for sublist in names])
        name=[sublist[0] for sublist in names if sublist[1]==max_loc]
    else: name=[]
    return name




#funstion to extract username from the post
def extract_info(raw_text, anon):
    timestamp='No date'
    username= 'Unsigned post'
    text1=raw_text.replace('‎','')
    t_split=text1.split('(UTC)') # this is mainly for property proposal, because after (UTC) it includes remaining punctuation
    t_connect1=[str(part)+'(UTC)' for part in t_split[ :-1]] # remove the final part with punctuaton
    t_connect='(UTC)'.join(t_connect1) # connect again the text
    
    if t_connect.endswith('(UTC)'):# if UTC in not in the text, the post does not have a signature or timestmap so the usernmae will be assigned as unsigned and the timestamp with no date
        peaces=re.split("(\d+:\d\d, \d+ \w+ \d\d\d\d \(UTC\))$", t_connect)
        if len(peaces)>1:
            timestamp=peaces[1]
            u_user_talk=find_username_User_talk(peaces[0])# step 1--- detect the pattern [[user talk:]] to find the username
            
            if len(u_user_talk)<1:# if step 1 do not work
                u_from_list= [name for name in username_lst if name in peaces[0]]# step 2 ----- check the usernmae list
                
                for name in u_from_list:
                    text=peaces[0].replace('(talk) ','')
                    u_peaces=re.split(name,text)
                    if len(u_peaces[-1])<10:
                        print(name)
                        return timestamp, name, anon
                if len(u_from_list)<1 or username== 'Unsigned post':# if step 1 and 2 do not work the username will be assigned as anonymous
                    anon += 1
                    username= 'Anonymous_username_'+str(anon)
                    print(username)
                    return timestamp, username, anon
                    
            else:
                print(u_user_talk)
                return timestamp, u_user_talk[0], anon
        else:
            print(username)
            return timestamp, username, anon    
    else:
        print(username)
        return timestamp, username, anon

    

 #---------RUN-----------


file_path='path_to_directory__files_for_process'
save_path='path_to_directory_to_save_files'

# load the list with usernames we have gathered during the dump process 
# read usernames 
my_file = open("path_to_username_list", "r", encoding='utf-8')
data = my_file.read()
# replacing end splitting the text 
# when newline ('\n') is seen.
username_lst = data.split("\n")
my_file.close()
# #separate usernmaes based on length because there are usernmaes with one character and confuse the correct detection
# usernames_short=[n for n in username_lst if len(n)<=2]
# usernames_long=[n for n in username_lst if len(n)>3]




anon=0 # this is to give names in case we can not recognise a username
for dump in os.listdir(str(file_path)):
    # dump='property_proposal_all.csv'
    df=pd.read_csv(str(file_path)+str(dump), encoding='utf-8')
    for row in range(len(df)):
        print(dump, row)
        # test="=== {{TranslateThis | anchor = en| en = Kinsky–Halm Catalogue <!-- |xx = property names in some other languages -->}} ==={{Property proposal|status			= 7161|description		= {{TranslateThis | en = Number in the Kinsky–Halm Catalogue of the works of Ludwig van Beethoven, which do not have opus numbers, or are fragmentary.<!-- | xx = descriptions in other languages --> }}|subject item           = {{Q|616043}}|infobox parameter	= [[en:catalogue:infobox musical composition]]<!-- Wikipedia infobox parameters, if any; ex: "population" in [[:en:template:infobox settlement]] -->|datatype		= external-id<!-- put datatype here (one of: item, external-id, string, media, coordinates, monolingual text, multilingual text, time, URL, number, geo-shape, tabular, lexeme, form, sense, mathematical expression, or a possibly new one) -->|domain			= <!-- entity type (item, property, lexeme, form, sense) and types of items that may bear this property; give QIDs if possible -->|allowed values		= \d+<!-- type of linked items (Q template or text), string pattern (regex if possible), list or range of allowed values, etc. -->|allowed units          = <!-- units that are allowed for values of this property -->|source			= https://en.wikipedia.org/wiki/WoO <!-- external reference URL, Wikipedia list article, etc. --><!-- you should provide 3 examples at least-->|example 1		= {{Q|8076022}} → 123 <!-- {{Q|1}} → value -->|example 2		= {{Q|166550}} → 4|example 3		= {{Q|166173}} → 6|example 4		= <!--  if convenient, include more examples by following the same pattern: |example ...		=  {{Q|1}} → value-->|planned use            = Add to Authority control<!-- what use do you plan to make of the property within the next month -->|number of ids          = <!-- for identifiers, the number of ids available from the source -->|expected completeness  = Q21873974 <!-- for identifiers, the coverage we can hope for in Wikidata: Q21873886 (always incomplete) or Q21873974 (eventually complete) -->|formatter URL		= <!-- for identifiers, URL pattern where $​1 replaces the value -->|external links         = <!-- for identifiers, search string to pass to sister projects' Special:LinkSearch pages, e.g. example.com  -->|see also               = <!-- other, related properties -->|filter			= <!-- sample: 7 digit number can be validated with edit filter [[Special:AbuseFilter/17]] -->|robot and gadget jobs	= <!-- Should or are bots or gadgets doing any task with this? (Checking other properties for consistency, collecting data, etc.) -->|subpage		= Werke ohne Opuszahl|topic			= authority control}}====Motivation====This is an important catalogue, which we should be able to reference, possibly as authority control.  I leave it to more experienced folk to consider whether there should be a generic WoO and a separate Kinsky–Halm Catalogue property. (E.G. http://www.raff.org/music/catalog/opus/woo.htm is a WoO catalogue for Joachim Raff.)All&nbsp;the&nbsp;best: ''[[Q23041480|Rich]]&nbsp;[[User talk:Rich Farmbrough|Farmbrough]]'',&nbsp;<small>12:41,&nbsp;4&nbsp;August&nbsp;2019&nbsp;(UTC).</small><br />===="
        # timestamp_row, username_row, anon = extract_info(test, anon)
        timestamp_row, username_row, anon = extract_info(df.loc[row,'post'], anon)
        save_df=pd.DataFrame({'post':[df.loc[row,'post']],'page_title':[df.loc[row,'page_title']],'thread_title':[df.loc[row,'thread_title']],'urername':[username_row],'timestamp':[timestamp_row]})
        save_df.to_csv(str(save_path)+str(dump),mode='a',index=False, header=not os.path.exists(str(save_path)+str(dump)))